In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI="sqlite:////home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
client._registry_uri

'sqlite:////home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlflow.db'

In [8]:
client.search_experiments()

[<Experiment: artifact_location='/home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlruns/1', creation_time=1715827735374, experiment_id='1', last_update_time=1715827735374, lifecycle_stage='active', name='my-new-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1715826061217, experiment_id='0', last_update_time=1715826061217, lifecycle_stage='active', name='Default', tags={}>]

In [22]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=20,
    order_by=["metrics.rmse ASC"] 
)

In [23]:
for run in runs:
    
    
    print(f"run-id : {run.info.run_id}, rmse : {run.data.metrics}" , end="\n\n")

run-id : 8646743db6e44246bb07d9b3eb854abd, rmse : {'rmse': 5.142775398711079}

run-id : 7c74af2537a8441b9077ba11bf71e673, rmse : {'rmse': 5.142775398711079}

run-id : 72b72fbad30545fc9f111e26049ec643, rmse : {'rmse': 5.142775398711079}

run-id : c0d860037ea34fe08243fc3f08b3d1c1, rmse : {'rmse': 5.206375104181443}

run-id : 1b0b7b27fa7041d8bb38da5141903382, rmse : {'rmse': 5.220721805773825}

run-id : 02fe30016e1245dcac632de05fc345ce, rmse : {'rmse': 5.257370988262138}

run-id : d870a9ea2b13429c8d4b59fe80322736, rmse : {'training_mean_squared_error': 22.657963668777757, 'training_mean_absolute_error': 3.1835728234622462, 'training_r2_score': 0.7217420406339894, 'training_root_mean_squared_error': 4.760038200348581, 'training_score': 0.7217420406339894, 'rmse': 6.028004137380311}

run-id : a3630fe8ae5f462b948d287b86da0a65, rmse : {'training_mean_squared_error': 24.381751155700425, 'training_mean_absolute_error': 3.4329526402097805, 'training_r2_score': 0.7005725482866817, 'training_root_

Registering Model

In [24]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [25]:
run_id="a3630fe8ae5f462b948d287b86da0a65"

model_uri=f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri,name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1715870886657, current_stage='None', description=None, last_updated_timestamp=1715870886657, name='nyc-taxi-regressor', run_id='a3630fe8ae5f462b948d287b86da0a65', run_link=None, source='/home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlruns/1/a3630fe8ae5f462b948d287b86da0a65/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [28]:
model_name="nyc-taxi-regressor"

lastest_versions=client.get_latest_versions(name=model_name)

for version in lastest_versions:
    print(f"version:{version.version}, stage:{version.current_stage}")

version:2, stage:Staging
version:3, stage:None


/tmp/ipykernel_459024/611107627.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  lastest_versions=client.get_latest_versions(name=model_name)


In [30]:
model_version=3
stage='Staging'

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage,
    archive_existing_versions=False

)

/tmp/ipykernel_459024/2695622314.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1715870886657, current_stage='Staging', description=None, last_updated_timestamp=1715871864414, name='nyc-taxi-regressor', run_id='a3630fe8ae5f462b948d287b86da0a65', run_link=None, source='/home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlruns/1/a3630fe8ae5f462b948d287b86da0a65/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [31]:
from datetime import datetime

date=datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transmitted to {stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1715870886657, current_stage='Staging', description='The model version 3 was transmitted to Staging on 2024-05-16', last_updated_timestamp=1715871977737, name='nyc-taxi-regressor', run_id='a3630fe8ae5f462b948d287b86da0a65', run_link=None, source='/home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlruns/1/a3630fe8ae5f462b948d287b86da0a65/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [34]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [35]:
df=read_dataframe("/home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/data/green_tripdata_2023-03.parquet")

In [36]:
runid="8646743db6e44246bb07d9b3eb854abd"
client.download_artifacts(run_id=runid,path="preprocessor",dst_path='.')

/home/kaustubh/mlops_zoomcamp/mlops/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/preprocessor'

In [37]:
import pickle

with open("preprocessor/preprocess.b",'rb') as f:
    dv=pickle.load(f)

In [38]:
X_test=preprocess(df,dv)

In [39]:
target="duration"
y_test=df[target].values

In [42]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/kaustubh/mlops_zoomcamp/mlops/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 478 ms, sys: 5.68 ms, total: 484 ms
Wall time: 501 ms


/home/kaustubh/mlops_zoomcamp/mlops/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.242637999409467}

In [43]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 35.9 ms, sys: 0 ns, total: 35.9 ms
Wall time: 34.9 ms


/home/kaustubh/mlops_zoomcamp/mlops/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/home/kaustubh/mlops_zoomcamp/mlops/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.283028000858574}

In [44]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_459024/1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1715859726205, current_stage='Production', description='', last_updated_timestamp=1715879911750, name='nyc-taxi-regressor', run_id='8646743db6e44246bb07d9b3eb854abd', run_link='', source='/home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlruns/1/8646743db6e44246bb07d9b3eb854abd/artifacts/models_mlfow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

In [45]:
# mlflow ui --port 8000 --backend-store-uri sqlite:////home/kaustubh/mlops_zoomcamp/Mlops-ZoomCamp/02_experiment-tracking/mlflow.db